In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(beaveR))
clustFile <- "../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold0/cluster_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
# tse <- buildTSE(treeTermFile = clustFile, coldata = coldata)

In [17]:
tse

class: TreeSummarizedExperiment 
dim: 254395 12 
metadata(8): tximetaInfo quantInfo ... txpsAnn infRepsScaled
assays(105): counts abundance ... mean variance
rownames(254395): ENST00000601814.5 ENST00000419160.3 ... Node254394
  Node254395
rowData names(1): meanInfRV
colnames(12): 1_1 2_1 ... 5_2 6_2
colData names(2): names condition
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):
rowLinks: a LinkDataFrame (254395 rows)
rowTree: 1 phylo tree(s) (138200 leaves)
colLinks: NULL
colTree: NULL

In [3]:
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
load(file=file.path(saveDir, "cutsCons.RData"))
load(file=file.path(saveDir, "tseCons.RData"))

In [2]:
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
load(file=file.path(saveDir, "tseCons.RData"))

In [4]:
temp <- tse
tree <- rowTree(temp)

In [5]:
sapply(cuts,length)

irv_height_0.05  irv_height_0.1    irv_height_1      lfcTPMDesc             LFC 
          76883           87912          119765          119765           84785 
         lfcTPM 
          86405

In [24]:
lambda <- c(0.05, 0.1, 1, 5, 10)
# for(i in lambda[4:5]) {
#     descSize <- sapply(Descendants(tree, seq(nrow(tse))), length)
#     metric <- (mcols(temp)[['meanInfRV']] + node.depth(tree, 2)*i) *descSize
#     objS <- solveForOptimalCut(tse, metVec = metric, type = 'min')
#     gammaN <- paste("gamma",i,sep="=")
#     cuts[[gammaN]] <- objS[["cut"]]
# }
names(cuts)[7:8] <- paste("irv", "height", lambda[4:5], sep = "_") 

In [16]:
length(cuts)

[1] 8

In [6]:
load("environment/brain_sim_nodtu/mode=gc_bias/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tse)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tse)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tse)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])


In [7]:
detNodes <- list()
negNodes <- list()

In [8]:
lambda <- c(0.05, 0.1, 1, 5, 10)

In [11]:
names(cuts)

[1] "irv_height_0.05" "irv_height_0.1"  "irv_height_1"    "lfcTPMDesc"     
[5] "LFC"             "lfcTPM"

In [25]:
for(i in lambda[4:5]) {
     ihname <- paste("irv", "height", i, sep = "_") 
     y <- temp[sort(cuts[[ihname]]),]
     y <- fishpond::scaleInfReps(y)
     y <- labelKeep(y)
     set.seed(1)
     y <- swish(y, x="condition")
#      dim(y)
     detNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
#         txps <- match(nodes[grep("ENST", nodes)], rownames(tse))
#         iNodes <- match(nodes[grep("Node", nodes)], rownames(tse))
         match(nodes, names(logFCNodes))
        
#         c(txps,iNodes)
    })
    negNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })
}

In [23]:
ihname
names(cuts)

[1] "irv_height_5"

[1] "irv_height_0.05" "irv_height_0.1"  "irv_height_1"    "lfcTPMDesc"     
[5] "LFC"             "lfcTPM"          "gamma=5"         "gamma=10"

In [12]:
metadata(temp)[["type"]]

NULL

In [26]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    print(n)
    stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
    colnames(stats) <- colnames(df)[c(3:5)]
    stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
    df <- rbind(df, stats)
}
print(df)

[1] "irv_height_0.05"
[1] "tp 0.989345032905045"
[1] "tp 0.958007303077726"
[1] "tp 0.915940663998116"
[1] "irv_height_0.1"
[1] "tp 0.989531840651352"
[1] "tp 0.962882096069869"
[1] "tp 0.925461741424802"
[1] "irv_height_5"
[1] "tp 0.997685950413223"
[1] "tp 0.963011031797534"
[1] "tp 0.920902341519132"
[1] "irv_height_10"
[1] "tp 0.990298797050834"
[1] "tp 0.956297799282035"
[1] "tp 0.919503348214286"
              Method Metric    FDR_0.01   FDR_0.05   FDR_0.10
fdr  irv_height_0.05    FDR  0.01065497  0.0419927 0.08405934
tpr  irv_height_0.05    TPR   0.6280088  0.7306545  0.7738214
fdr1  irv_height_0.1    FDR  0.01046816  0.0371179 0.07453826
tpr1  irv_height_0.1    TPR   0.6233742  0.7270562  0.7710203
fdr2    irv_height_5    FDR  0.00231405 0.03698897 0.07909766
tpr2    irv_height_5    TPR   0.3602722  0.7086069  0.7699654
fdr3   irv_height_10    FDR 0.009701203  0.0437022 0.08049665
tpr3   irv_height_10    TPR   0.5930057  0.7118624   0.765772


In [29]:
assays(tse)

List of length 105
names(105): counts abundance length infRep1 ... infRep100 mean variance

In [27]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
seBrainSim <- tximeta(coldata)
yTermThrNS <- seBrainSim[rownames(tse)[1:l],]
groupsClust <- parseClustFile(file.path(dir, "old", "1_1", "clusters.txt"), yTermThrNS)
mInds <- seq(nrow(yTermThrNS) + length(groupsClust))
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)

rownames(yAggTermThrNS)[l+1:length(groupsClust)] <- paste("Node", as.character(l+1:length(groupsClust)),sep="")
metadata(yAggTermThrNS)[["txpsAnn"]] <- rowData(yTermThrNS)
yAggTermThrNS <- yAggTermThrNS[-unlist(groupsClust),]
dim(yAggTermThrNS)

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Homo sapiens - release 26 ]

loading existing TxDb created: 2022-08-19 22:12:59

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

loading existing transcript ranges created: 2022-08-19 22:13:01



[1] 129988     12

In [31]:
head(assays(yAggTermThrNS)[["length"]])

,1_1,2_1,3_1,4_1,5_1,6_1,1_2,2_2,3_2,4_2,5_2,6_2
ENST00000601814.5,452.528,452.896,447.910,447.290,449.833,452.545,453.011,452.652,448.118,447.572,450.137,453.316
ENST00000419160.3,425.819,424.925,418.175,421.895,416.115,417.960,426.296,424.830,418.442,421.851,416.367,418.479
ENST00000416931.1,171.000,171.018,171.523,171.000,171.653,174.336,171.557,171.525,171.646,171.000,171.891,174.675
ENST00000457540.1,847.944,849.914,851.183,843.994,856.503,867.309,849.478,850.894,851.341,843.980,857.007,868.483
ENST00000414273.1,1362.011,1358.358,1350.463,1361.697,1337.770,1355.478,1364.644,1360.710,1350.887,1359.909,1337.996,1356.260
ENST00000514057.1,482.604,482.231,482.289,482.576,480.867,488.015,483.840,483.360,482.505,482.193,481.241,488.598


In [38]:
length(groupsClust[[1]])
groupsClust[[2]]

[1] 2

[1] 4653 4652

In [149]:
groupsClust[[3]]

[1] 97036 97035

In [150]:
Ancestors(tree, groupsClust[[3]][1])

[1] 218826 218825 218824 218823 218822 218821 218820 218819 218818 218817
[11] 218816 218815 218814 218813 218812 218811 218810 218809 138201

In [243]:
i <- 218826
Descendants(tree, i)

[[1]]
[1] 97035 97036

In [244]:
assays(tse)[["counts"]][i,]
assays(tse)[["length"]][i,]

1_1      2_1      3_1      4_1      5_1      6_1      1_2      2_2 
1382.811 1185.555 1295.675 1364.115 1272.772 1638.383  393.310  317.713 
     3_2      4_2      5_2      6_2 
 308.338  389.385  412.740  711.706

1_1      2_1      3_1      4_1      5_1      6_1      1_2      2_2 
1875.895 1806.603 2026.748 1864.610 1828.366 1902.031 1876.803 1897.461 
     3_2      4_2      5_2      6_2 
2030.948 1956.667 1860.878 1907.937

In [245]:
assays(yAggTermThrNS)[["counts"]]["Node138203",]
assays(yAggTermThrNS)[["length"]]["Node138203",]

1_1      2_1      3_1      4_1      5_1      6_1      1_2      2_2 
1382.811 1185.555 1295.675 1364.115 1272.772 1638.383  393.310  317.713 
     3_2      4_2      5_2      6_2 
 308.338  389.385  412.740  711.706

1_1      2_1      3_1      4_1      5_1      6_1      1_2      2_2 
1875.895 1806.603 2026.748 1864.610 1828.366 1902.031 1876.803 1897.461 
     3_2      4_2      5_2      6_2 
2030.948 1956.667 1860.878 1907.937

In [96]:
dim(yAggTermThrNS)[1] - length(unlist(groupsClust))-1

[1] 116375

In [126]:
"Node138212" %in% rownames(assays(yAggTermThrNS)[["length"]])

[1] TRUE

In [118]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
seBrainSim <- tximeta(coldata)
yTermThrNS <- seBrainSim[rownames(tse)[1:l],]
groupsClust <- parseClustFile(file.path(dir, "old", "1_1", "clusters.txt"), yTermThrNS)
mInds <- seq(nrow(yTermThrNS) + length(groupsClust))
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)

[1] 138200

In [242]:
rownames(yAggTermThrNS)[l+1:length(groupsClust)] <- paste("Node", as.character(l+1:length(groupsClust)),sep="")
metadata(yAggTermThrNS)[["txpsAnn"]] <- rowData(yTermThrNS)
yAggTermThrNS <- yAggTermThrNS[-unlist(groupsClust),]
dim(yAggTermThrNS)

[1] 129988     12

In [249]:
# aggCountsNodes <- computeOAggNodesU(groupsClust, mInds, sim.counts.mat)
# logFCNodesTerm <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
# names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)] <- paste("Node", names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)], sep="")

yAggTermThrNS <- fishpond::scaleInfReps(yAggTermThrNS)
yAggTermThrNS <- labelKeep(yAggTermThrNS)
set.seed(1)
yAggTermThrNS <- swish(yAggTermThrNS, x="condition")

ERROR: Error in fishpond::scaleInfReps(yAggTermThrNS): inferential replicates already scaled


In [250]:
set.seed(1)
yAggTermThrNS <- swish(yAggTermThrNS, x="condition")

In [214]:
rownames(yAggTermThrNS)[l+1:length(groupsClust)] <- paste("Node", as.character(l+1:length(groupsClust)),sep="")
metadata(yAggTermThrNS)[["txpsAnn"]] <- rowData(yTermThrNS)
yAggTermThrNS <- yAggTermThrNS[-unlist(groupsClust),]

In [246]:
1

[1] 1

In [248]:
tail(assays(yAggTermThrNS)[["infRep1"]])

,1_1,2_1,3_1,4_1,5_1,6_1,1_2,2_2,3_2,4_2,5_2,6_2
Node143595,1897.348,1842.470,2090.076,1703.232,2309.254,2423.907,2431.316,2260.820,1459.841,1986.137,2064.201,2340.857
Node143596,2862.412,2778.919,2374.364,2963.617,3140.346,2931.553,2511.615,2864.257,3054.326,2924.688,3288.171,2478.713
Node143597,3485.009,3146.092,3984.090,4313.931,3304.299,3960.503,4815.902,4533.252,4067.127,4657.007,4100.220,3614.558
Node143598,16319.709,16329.522,16742.793,15440.213,20586.165,18217.234,3601.249,4559.657,3799.783,4417.311,3954.467,4285.702
Node143599,4754.903,4574.026,4262.188,4962.590,4793.641,4214.605,4678.849,3863.630,3692.468,4516.303,4195.454,5154.811
Node143600,1252.032,1575.908,1642.468,1633.856,1369.179,1339.731,1632.108,1578.187,1850.237,1869.456,1343.913,1255.110


In [237]:
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)

In [230]:
scaleInfReps <- function (y, lengthCorrect = TRUE, meanDepth = NULL, sfFun = NULL, 
    minCount = 10, minN = 3, saveMeanScaled = FALSE, quiet = FALSE) 
{
    if (!interactive()) {
        quiet <- TRUE
    }
    if (!is.null(metadata(y)$infRepsScaled)) {
        if (metadata(y)$infRepsScaled) 
            stop("inferential replicates already scaled")
    }
    infRepIdx <- grep("infRep", assayNames(y))
    infReps <- assays(y)[infRepIdx]
    counts <- assays(y)[["counts"]]
    length <- assays(y)[["length"]]
    nreps <- length(infReps)
    if (is.null(meanDepth) & !is(sfFun, "numeric")) {
        meanDepth <- exp(mean(log(colSums(counts))))
    }
    means <- matrix(nrow = nrow(y), ncol = nreps)
    if (is.null(length)) {
        if (lengthCorrect) {
            if (!quiet) 
                message("not correcting for feature length (lengthCorrect=FALSE)")
        }
        lengthCorrect <- FALSE
    }
#     for (k in seq_len(nreps)) {
    for (k in seq(5)) {    
        if (!quiet) 
            svMisc::progress(k, max.value = nreps, init = (k == 
                1), gui = FALSE)
        if (lengthCorrect) {
            length <- length/exp(rowMeans(log(length)))
            cts <- infReps[[k]]/length
        }
        else {
            cts <- infReps[[k]]
        }
        if (!is(sfFun, "numeric")) {
            cts <- t(t(cts)/colSums(cts)) * meanDepth
            use <- rowSums(infReps[[k]] >= minCount) >= minN
            if (is.null(sfFun)) {
                loggeomeans <- rowMeans(log(cts[use, ]))
                sf <- apply(cts[use, ], 2, function(s) {
                  exp(median((log(s) - loggeomeans)[is.finite(loggeomeans)]))
                })
            }
            else if (is(sfFun, "function")) {
                sf <- sfFun(cts)
            }
        }
        else {
            sf <- sfFun
        }
        print(sf)
        infReps[[k]] <- t(t(cts)/sf)
#         print(head(infReps[[k]]))
        means[, k] <- rowMeans(infReps[[k]])
    }
    if (!quiet) 
        message("")
    assays(y)[grep("infRep", assayNames(y))] <- infReps
    mcols(y)$log10mean <- log10(rowMeans(means) + 1)
    metadata(y)$infRepsScaled <- TRUE
    if (saveMeanScaled) {
        infRepsArray <- abind::abind(as.list(infReps), along = 3)
        meanScaled <- apply(infRepsArray, 1:2, mean)
        assays(y)[["meanScaled"]] <- meanScaled
    }
    y
}

In [239]:
yAggTermThrNS <- dd

In [240]:
sum(is.na(assays(dd)[["length"]]))

[1] 0

In [251]:
load(file.path(saveDir, "cutsMean.RData"))